Human in the loop : Interrupt

In [17]:
from IPython.display import Image, display
import operator
from typing import Annotated, List , Literal, TypedDict
from langgraph.graph import END, START, StateGraph
from langgraph.types import Command, interrupt

In [18]:
# generally interrupt helps us to pause graph execution at specific points to gert human input for interactive workflow

from langgraph.checkpoint.memory import InMemorySaver

memory = InMemorySaver()

config = {
    "configurable":{
        "thread_id" : "1"
    }
}

In [19]:
class State(TypedDict) :
    nlist : Annotated[list[str], operator.add]

In [20]:
def node_a(state : State) -> Command[Literal["b","c",END]] :
    print("Entered 'a' Code")
    select = state["nlist"][-1]
    
    if select == "b":
        next_node = "b"
    elif select == "c":
        next_node = "c"
    elif select == "q":
        next_node = END
    else :
        admin = interrupt(f"Unexpected input '{select}")
        print(admin)
        
        if admin == "continue" :
            next_node = "b"
        else :
            next_node = END
            select = "q"
    
    return Command(
        update = State(nlist = [select]),
        goto = next_node
    )

def node_b(state : State) -> State :
    return(State(nlist = ["B"]))

def node_c(state : State) -> State :
    return(State(nlist = ["C"]))

In [21]:
builder = StateGraph(State)

# adding nodes

builder.add_node("a", node_a)
builder.add_node("b", node_b)
builder.add_node("c", node_c)

# adding edges 

builder.add_edge(START, "a")
builder.add_edge("b", END)
builder.add_edge("c", END)

# compile

graph = builder.compile(checkpointer = memory)
 

In [24]:
while True :
    user = input('b,c or q to quit : ')
    input_state = State(nlist = [user])
    
    result = graph.invoke(input_state, config)
    print(result)
    break
    if result['nlist'][-1] == "q" :
        print("quit")
        break
    

Entered 'a' Code
{'nlist': ['v', 'b', 'b', 'B', 'v'], '__interrupt__': [Interrupt(value="Unexpected input 'v", id='9e3d032ee70cebc0fb32b6ab887ccb74')]}


In [25]:
# we can see there is an interrupt so we need to handle this interrupt in graph
while True :
    user = input('b,c or q to quit : ')
    input_state = State(nlist = [user])
    
    result = graph.invoke(input_state, config)
    if '__interrupt__' in result :
        print(f"Interrupt: {result}")
        msg = result['__interrupt__'][-1].value
        print(msg)
        
        human = input(f"\n{msg}: ")
        
        human_response = Command(
            resume = human
        )
        
        result = graph.invoke(human_response,config)
    if result['nlist'][-1] == "q" :
        print("quit")
        break

Entered 'a' Code
Interrupt: {'nlist': ['v', 'b', 'b', 'B', 'v', 'v'], '__interrupt__': [Interrupt(value="Unexpected input 'v", id='a6c595ea8327d462ffc09ebd31a03b76')]}
Unexpected input 'v
Entered 'a' Code
sdgsf
quit
